In [ ]:
# Celda 1: Setup
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

sys.path.insert(0, str(Path.cwd().parent))

from src.query_parser.parser import QueryParser
from src.property_analysis.text_analyzer import PropertyTextAnalyzer
from src.property_analysis.schemas import QueryRequirement

# Cargar datos
data_dir = Path("../data/raw")
all_properties = []
for json_file in data_dir.glob("fotocasa_*.json"):
    with open(json_file) as f:
        data = json.load(f)
        all_properties.extend(data['properties'])

print(f"📊 Loaded {len(all_properties)} properties")

In [ ]:
# Celda 2: Queries de prueba
test_queries = [
    "Local con entrada independiente",
    "Piso luminoso con terraza",
    "Local comercial Barcelona",
    "Vivienda 3 habitaciones",
    "Local con parking incluido"
]

print("🔍 Test Queries:")
for i, q in enumerate(test_queries, 1):
    print(f"{i}. {q}")

In [ ]:
# Celda 3: Analizar todas las propiedades (1 vez)
analyzer = PropertyTextAnalyzer(backend="api")

print("🔍 Analyzing all properties...")
analyses = analyzer.analyze_batch(all_properties[:50], generate_embeddings=True)
print(f"✅ Analyzed {len(analyses)} properties")

# Guardar para reutilizar
import pickle
with open('../data/cache/analyses.pkl', 'wb') as f:
    pickle.dump(analyses, f)

In [ ]:
# Celda 4: Cargar analyses (si ya las tienes)
import pickle
with open('../data/cache/analyses.pkl', 'rb') as f:
    analyses = pickle.load(f)
print(f"✅ Loaded {len(analyses)} analyses")

In [ ]:
# Celda 5: Evaluar cada query
results_by_query = {}

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    
    matches = []
    for analysis in analyses:
        match = analyzer.match_against_query(analysis, query)
        matches.append(match)
    
    # Ordenar
    matches.sort(key=lambda m: m.final_score, reverse=True)
    
    # Top 5
    print(f"\nTop 5 matches:")
    for i, match in enumerate(matches[:5], 1):
        prop = next(p for p in all_properties if p['id'] == match.property_id)
        print(f"\n{i}. {prop['id']} - Score: {match.final_score:.2f}")
        print(f"   {prop['location']} | {prop['price']:,}€")
        print(f"   Matched features: {len(match.matched_features)}")
    
    results_by_query[query] = matches

print("\n✅ Evaluation complete")

In [ ]:
# Celda 6: Visualización de scores
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (query, matches) in enumerate(results_by_query.items()):
    scores = [m.final_score for m in matches]
    
    axes[idx].hist(scores, bins=20, alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'Query {idx+1}\n{query[:30]}...', fontsize=10)
    axes[idx].set_xlabel('Score')
    axes[idx].set_ylabel('Count')
    axes[idx].axvline(0.5, color='red', linestyle='--', label='Good threshold')
    axes[idx].legend()

plt.tight_layout()
plt.savefig('../reports/figures/score_distributions.png', dpi=300)
plt.show()

print("✅ Saved to reports/figures/score_distributions.png")

In [ ]:
# Celda 7: Análisis de features más comunes
all_features = {}
for analysis in analyses:
    for feat in analysis.detected_features:
        if feat.confidence >= 0.5:
            all_features[feat.name] = all_features.get(feat.name, 0) + 1

# Top 20 features
top_features = sorted(all_features.items(), key=lambda x: x[1], reverse=True)[:20]

plt.figure(figsize=(12, 6))
features, counts = zip(*top_features)
plt.barh(features, counts)
plt.xlabel('Count')
plt.title('Top 20 Most Common Features Detected')
plt.tight_layout()
plt.savefig('../reports/figures/common_features.png', dpi=300)
plt.show()

print(f"✅ Most common features:")
for feat, count in top_features[:10]:
    print(f"   {feat}: {count}")

In [ ]:
# Celda 8: Matriz de correlación Query-Property
# ¿Qué queries matchean mejor?

query_scores = []
for query, matches in results_by_query.items():
    avg_score = sum(m.final_score for m in matches) / len(matches)
    top5_avg = sum(m.final_score for m in matches[:5]) / 5
    good_matches = sum(1 for m in matches if m.final_score >= 0.5)
    
    query_scores.append({
        'query': query[:30],
        'avg_score': avg_score,
        'top5_avg': top5_avg,
        'good_matches': good_matches
    })

df_queries = pd.DataFrame(query_scores)
print(df_queries)